# Introduction #

Thanks to Kaggle mini-courses on computer vision for getting me started on this.
My first trained CNN to classify the card images heavily inspired by Francesco Marazz's CNN (https://www.kaggle.com/fmarazzi/baseline-keras-cnn-roc-fast-10min-0-925-lb) for the Histopathologic Cancer Detection competition (https://www.kaggle.com/c/histopathologic-cancer-detection).

My objective here is to build my first image classifier using Keras. Rare Pokemon cards are highly sought after by collectors, with some even reaching re-sale prices of hundreds of thousands of dollars. It would be important for collectors to tell a genuine card from fake card to avoid getting duped. Experienced collectors are able to tell the different between a genuine and fake card by looking at the features of the back of the card alone, but this might not be so obvious for newcomers. The image classifier described below will aim to classify, with high accuracy (>95%), Pokemon cards as genuine or fake based on the back visuals of the card alone. In future work, the model can be made more robust by including more counterfeit variations, folded, torn, new and old cards in the training, validation and test sets.

# Load Data #

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
print(os.listdir("../input"))

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [3]:
base_tile_dir = '../input/real-and-fake-pokemon-cards/train'
df = pd.DataFrame({'path': glob(os.path.join(base_tile_dir,'*.JPG'))})
df['id'] = df.path.map(lambda x: x.split('/')[4].split(".")[0])
labels = pd.read_csv("../input/real-and-fake-pokemon-cards/train_labels.csv")
labels['id'] = labels['id'].astype('str')
labels = labels.sort_values(by=['id'], ignore_index = True)
df['id'] = df['id'].astype('str')
df = df.sort_values(by=['id'], ignore_index = True)
df_data = pd.merge(labels, df, on="id")


# Data Visualization #

In [4]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def readImage(path):
    # OpenCV reads the image in bgr format by default
    bgr_img = cv2.imread(path)
    # We flip it to rgb for visualization purposes
    b,g,r = cv2.split(bgr_img)
    rgb_img = cv2.merge([r,g,b])
    return rgb_img

In [5]:
# random sampling
shuffled_data = shuffle(df_data)

fig, ax = plt.subplots(2,5, figsize=(20,8))
fig.suptitle('Real and Fake Pokemon Cards',fontsize=20)
# Real
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 1]['id'][:5]):
    path = os.path.join(base_tile_dir, idx)
    ax[0,i].imshow(readImage(path + '.JPG'))
ax[0,0].set_ylabel('Real Cards', size='large')
# Fake
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 0]['id'][:5]):
    path = os.path.join(base_tile_dir, idx)
    ax[1,i].imshow(readImage(path + '.JPG'))
ax[1,0].set_ylabel('Fake Cards', size='large')


# Split X and y in train/test and build folders


In [6]:
# Set the id as the index in df_data
df_data.set_index('id', inplace=True)
df_data.head()

In [7]:
# train_test_split # stratify=y creates a balanced validation set.
y = df_data['label']
df_train, df_val = train_test_split(df_data, test_size=0.10, random_state=101, stratify=y)

In [8]:
# Create directories
train_path = 'base_dir/train'
valid_path = 'base_dir/valid'
test_path = '../input/real-and-fake-pokemon-cards/test'
for fold in [train_path, valid_path]:
    for subf in ["0", "1"]:
        os.makedirs(os.path.join(fold, subf))

In [9]:
for image in df_train.index.values:
    # the id in the csv file does not have the .JPG extension therefore we add it here
    fname = image + '.JPG'
    label = str(df_data.loc[image,'label']) # get the label for a certain image
    src = os.path.join('../input/real-and-fake-pokemon-cards/train', fname)
    dst = os.path.join(train_path, label, fname)
    shutil.copyfile(src, dst)

for image in df_val.index.values:
    fname = image + '.JPG'
    label = str(df_data.loc[image,'label']) # get the label for a certain image
    src = os.path.join('../input/real-and-fake-pokemon-cards/train', fname)
    dst = os.path.join(valid_path, label, fname)
    shutil.copyfile(src, dst)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 256
num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 32
val_batch_size = 32

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
                            horizontal_flip=True,
                            vertical_flip=True)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='binary')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='binary')

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='binary',
                                        shuffle=False)

# Define the model 
**Model structure (optimizer: Adam):**

* In 
* [Conv2D*3 -> MaxPool2D -> Dropout] x3 --> (filters = 16, 32, 64)
* Flatten 
* Dense (256) 
* Dropout 
* Out

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam

kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.5

model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(1, activation = "sigmoid"))

# Compile the model
model.compile(Adam(0.01), loss = "binary_crossentropy", metrics=["accuracy"])

In [12]:
import tensorflow as tf
dot_img_file = '/tmp/model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

# Train

In [13]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.1)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=50,
                   callbacks=[reducel, earlystopper])

In [17]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

# make a prediction
y_pred_keras = model.predict_generator(test_gen, steps=len(df_val), verbose=1)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_gen.classes, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)
auc_keras

# Plot ROC Curve

In [18]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# Load test data and predict


In [21]:
base_test_dir = '../input/real-and-fake-pokemon-cards/test'
test_files = glob(os.path.join(base_test_dir,'*.JPG'))
submission = pd.DataFrame()
file_batch = 78
max_idx = len(test_files)
for idx in range(0, max_idx, file_batch):
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})
    test_df['id'] = test_df.path.map(lambda x: x.split('/')[4].split(".")[0])
    test_df['image'] = test_df['path'].map(imread)
    K_test = np.stack(test_df["image"].values)
    K_test = (K_test - K_test.mean()) / K_test.std()
    predictions = model.predict(K_test)
    predictions = list(map(lambda x: 0 if x<0.5 else 1, predictions)) # get binary values predictions with 0.5 as thresold
    test_df['predict'] = predictions
    submission = pd.concat([submission, test_df[["id", "predict"]]])
submission.head()
df_test = pd.read_csv('../input/real-and-fake-pokemon-cards/test_labels.csv')
df_test['id'] = df_test['id'].astype('str')
check_results = pd.merge(submission, df_test, on='id')

In [22]:
check_results

In [23]:
from sklearn.metrics import accuracy_score
model_accuracy = accuracy_score(check_results['label'],check_results['predict'])
print('Accuracy =', model_accuracy)

In [ ]:
check_results.to_csv("results.csv", index = False, header = True)